# Eliminación de nodos en Árboles B

La eliminación de nodos en los Árboles B es un proceso delicado que asegura que el árbol siga cumpliendo con sus propiedades fundamentales después de la eliminación. Los Árboles B son estructuras de datos de tipo árbol que generalizan los Árboles Binarios de Búsqueda (ABB), permitiendo que un nodo tenga más de dos hijos. Esto los hace especialmente útiles para sistemas que requieren lecturas y escrituras de grandes volúmenes de datos en almacenamiento secundario, como bases de datos y sistemas de archivos.

## Proceso de Eliminación

Eliminar un nodo en un Árbol B implica varios pasos para mantener las propiedades del árbol. A continuación, describimos los pasos generales involucrados:

- **Buscar el nodo a eliminar:** Primero, se debe buscar el nodo que contiene el valor a eliminar, comenzando desde la raíz y avanzando hacia las hojas, eligiendo el hijo adecuado basado en el valor de las claves.
- **Eliminar el nodo:**
  - Si el nodo es una **hoja**, simplemente se elimina la clave del nodo.
  - Si el nodo es **interno**, se pueden seguir dos estrategias:
    - **Sustitución por predecesor:** Reemplazar la clave a eliminar con su predecesor inmediato (la máxima clave del subárbol izquierdo).
    - **Sustitución por sucesor:** Reemplazar la clave a eliminar con su sucesor inmediato (la mínima clave del subárbol derecho).
- **Rebalancear el árbol:** Después de la eliminación, si algún nodo viola las propiedades de los Árboles B (como tener menos claves de las permitidas), se debe rebalancear el árbol. Esto puede implicar:
  - **Fusionar nodos:** Si un nodo y su hermano tienen menos claves de las permitidas, se pueden fusionar en un solo nodo.
  - **Redistribuir claves:** Si un nodo tiene menos claves de las permitidas, pero un hermano tiene más de las mínimas requeridas, se pueden redistribuir las claves entre ellos.

## Implementación en Python

In [1]:
# Código utilitario
from src.visualization import visualize_b_tree
# ver src/BTree.py
from src.BTree import BTree

In [2]:
def delete(self, k):
    i = 0
    while i < len(self.keys) and k > self.keys[i]:
        i += 1

    if i < len(self.keys) and self.keys[i] == k:
        if self.leaf:
            # Caso 1: La clave está en el nodo y es una hoja
            self.keys.pop(i)
        else:
            # Caso 2: La clave está en el nodo y es un nodo interno
            if len(self.children[i].keys) >= self.t:
                self.keys[i] = self.children[i].get_pred(i)
                self.children[i].delete(self.keys[i])
            elif len(self.children[i + 1].keys) >= self.t:
                self.keys[i] = self.children[i + 1].get_succ(i)
                self.children[i + 1].delete(self.keys[i])
            else:
                self.merge(i)
                self.children[i].delete(k)
    else:
        if self.leaf:
            # La clave no está presente y estamos en una hoja
            return
        flag = (i == len(self.keys))
        if len(self.children[i].keys) < self.t:
            self.fill(i)
        if flag and i > len(self.keys):
            self.children[i - 1].delete(k)
        else:
            self.children[i].delete(k)


def fill(self, idx):
    """
    Llena el hijo del nodo actual en el índice 'idx' que tiene menos de 't-1' claves.
    """
    # Si el hermano anterior (izquierdo) tiene más de t-1 claves, toma una clave de ese hermano
    if idx != 0 and len(self.children[idx - 1].keys) >= self.t:
        self.borrow_from_prev(idx)
    # Si el hermano siguiente (derecho) tiene más de t-1 claves, toma una clave de ese hermano
    elif idx != len(self.keys) and len(self.children[idx + 1].keys) >= self.t:
        self.borrow_from_next(idx)
    # Si el hijo en idx es el último hijo, fusiona con su hermano izquierdo
    # De lo contrario, fusiona con su hermano derecho
    else:
        if idx != len(self.keys):
            self.merge(idx)
        else:
            self.merge(idx - 1)

def borrow_from_prev(self, idx):
    """
    Toma una clave del hijo idx-1 y la mueve al hijo idx.
    """
    child = self.children[idx]
    sibling = self.children[idx - 1]

    # La última clave del hermano previo sube al padre y la clave del padre baja al hijo actual
    child.keys.insert(0, self.keys[idx - 1])
    if not child.leaf:
        child.children.insert(0, sibling.children.pop())
    self.keys[idx - 1] = sibling.keys.pop()


def borrow_from_next(self, idx):
    """
    Toma una clave del hijo idx+1 y la mueve al hijo idx.
    """
    child = self.children[idx]
    sibling = self.children[idx + 1]

    # La primera clave del hermano siguiente baja al padre y la clave del padre sube al hijo actual
    child.keys.append(self.keys[idx])
    if not child.leaf:
        child.children.append(sibling.children.pop(0))
    self.keys[idx] = sibling.keys.pop(0)


def merge(self, idx):
    """
    Fusiona el hijo en 'idx' con su hermano. La clave del padre en 'idx' baja al nuevo nodo fusionado.
    """
    child = self.children[idx]
    sibling = self.children[idx + 1]
    child.keys.append(self.keys.pop(idx))

    # Mover las claves y los hijos del hermano derecho al hermano izquierdo
    child.keys.extend(sibling.keys)
    if not child.leaf:
        child.children.extend(sibling.children)

    # Eliminar la referencia al hermano derecho
    self.children.pop(idx + 1)


# Extender la clase BTree con el nuevo método
BTree.delete = delete
BTree.fill = fill
BTree.borrow_from_prev = borrow_from_prev
BTree.borrow_from_next = borrow_from_next
BTree.merge = merge

## Pruebas exhaustivas

In [3]:
def prueba_insercion_y_eliminacion():
    grados = [3, 4]  # Prueba con diferentes grados para el árbol
    for grado in grados:
        btree = BTree(grado)
        print(f"\nÁrbol B con grado mínimo {grado}")

        # Insertar claves para tener contenido en el árbol
        claves_para_insertar = [10, 20, 5, 6, 12, 30, 7, 17]
        for clave in claves_para_insertar:
            btree.insert(clave)
        print("Claves insertadas:", claves_para_insertar)

        # Eliminar una clave que cause un préstamo simple
        btree.delete(6)
        print("Clave 6 eliminada (prueba de préstamo).")

        # Eliminar una clave que cause una fusión
        btree.delete(7)
        print("Clave 7 eliminada (prueba de fusión).")

        # Intentar eliminar una clave que no existe (debe manejar sin errores)
        btree.delete(100)
        print("Intento de eliminar clave 100 (no existe).")

        # Eliminar múltiples claves para probar varios préstamos y fusiones
        for clave in [5, 12, 17]:
            btree.delete(clave)
            print(
                f"Clave {clave} eliminada (prueba de múltiples operaciones).")

        # Eliminar claves hasta que el árbol esté vacío
        for clave in claves_para_insertar:
            btree.delete(clave)
        print("Árbol vaciado.")


def prueba_arbol_vacio():
    btree = BTree(3)
    print("\nPrueba con árbol vacío")
    # Intentar eliminar en un árbol vacío
    btree.delete(15)
    print("Intento de eliminar clave 15 en árbol vacío.")


def prueba_arbol_una_clave():
    btree = BTree(2)
    print("\nPrueba con árbol de una sola clave")
    btree.insert(10)
    # Eliminar la única clave en el árbol
    btree.delete(10)
    print("Clave 10 eliminada en árbol de una sola clave.")


# Ejecutar pruebas
prueba_insercion_y_eliminacion()
prueba_arbol_vacio()
prueba_arbol_una_clave()

Nodo BTree inicializado como hoja
Árbol B inicializado con grado mínimo 3

Árbol B con grado mínimo 3
Llave insertada en nodo hoja.
Llave insertada: 10
Llave insertada en nodo hoja.
Llave insertada: 20
Llave insertada en nodo hoja.
Llave insertada: 5
Llave insertada en nodo hoja.
Llave insertada: 6
Llave insertada en nodo hoja.
Llave insertada: 12
Nodo BTree inicializado
Nodo BTree inicializado como hoja
Nodo hijo dividido en dos nodos.
Llave insertada en nodo hoja.
Navegando hacia hijo para continuar inserción.
Llave insertada: 30
Llave insertada en nodo hoja.
Navegando hacia hijo para continuar inserción.
Llave insertada: 7
Llave insertada en nodo hoja.
Navegando hacia hijo para continuar inserción.
Llave insertada: 17
Claves insertadas: [10, 20, 5, 6, 12, 30, 7, 17]


AttributeError: 'BTree' object has no attribute 'keys'

## Complejidad del Algoritmo

- **Complejidad del tiempo:** La eliminación en un Árbol B tiene una complejidad de tiempo promedio de \(O(\log n)\), donde \(n\) es el número de claves en el árbol. Esto se debe a que el proceso de búsqueda, eliminación y rebalanceo se realiza a lo largo de la altura del árbol, que es logarítmica respecto al número de claves.
- **Complejidad del espacio:** La complejidad del espacio de un Árbol B es \(O(n)\), donde \(n\) es el número de claves en el árbol. Sin embargo, la operación de eliminación en sí misma utiliza un espacio adicional constante, por lo que su complejidad espacial es \(O(1)\), excluyendo el espacio ocupado por el árbol.   